In [1]:
!pip install numpy
!pip install plotly
!pip install pandas
!pip install scipy
!pip install matplotlib
!pip install sklearn
!pip install seaborn
!pip install interpolate

In [8]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns

from scipy import interpolate
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn import linear_model, preprocessing, metrics
from sklearn.linear_model import LinearRegression
from scipy.interpolate import interp1d

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [9]:
df = pd.read_csv("test_well.csv")
df.head()

,DT_HOUR,EXT_DATA,LIQ_RATE,OIL_RATE,WATER_CUT,PLIN,QGAS,FREQ_HZ,ACTIV_POWER,PED_T,PINP,I_B,I_A,I_C,U_AB,U_BC,U_CA,U_OTP,COS_PHI
0,2021-03-21 00:00:00,0,NaN,NaN,NaN,NaN,NaN,NaN,67.925,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-03-21 00:19:23,0,NaN,NaN,NaN,16.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-03-21 00:19:26,0,NaN,NaN,NaN,NaN,NaN,40.0,NaN,0.0,37.17,29.5,29.4,29.6,396.0,399.0,399.0,NaN,71.0
3,2021-03-21 00:20:00,0,NaN,NaN,NaN,NaN,NaN,NaN,67.920,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-03-21 00:38:32,0,NaN,NaN,NaN,17.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df = df.drop(["EXT_DATA", "PED_T", "U_OTP","OIL_RATE"], axis=1)

In [11]:
df["DT_HOUR"] = pd.to_datetime(df['DT_HOUR'], format='%Y-%m-%dT')

In [13]:
df["date"] = df['DT_HOUR'].apply(lambda x: x.strftime('%Y-%m-%d'))
df["date"]=pd.to_datetime(df["date"]).astype('int64')

In [14]:
df = df.groupby("date").agg("mean")
df

,LIQ_RATE,WATER_CUT,PLIN,QGAS,FREQ_HZ,ACTIV_POWER,PINP,I_B,I_A,I_C,U_AB,U_BC,U_CA,COS_PHI
date,,,,,,,,,,,,,,
1616284800000000000,388.84,87.46,17.755278,1853.77,40.0,68.041444,37.168472,29.459722,29.461111,29.552778,395.416667,398.861111,398.958333,71.000000
1616371200000000000,390.57,88.04,17.674028,1809.27,40.0,68.106472,37.155556,29.462500,29.463889,29.563889,393.916667,399.166667,397.638889,71.000000
1616457600000000000,395.10,87.95,17.800694,1822.53,40.0,68.024792,37.147778,29.445833,29.441667,29.537500,395.486111,400.666667,398.083333,71.000000
1616544000000000000,389.34,88.02,17.786528,1855.33,40.0,68.118278,37.144306,29.451389,29.444444,29.534722,395.138889,400.083333,397.805556,71.000000
1616630400000000000,391.28,88.22,17.830278,1827.74,40.0,68.120028,37.135000,29.454167,29.445833,29.550000,396.805556,399.916667,398.111111,71.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1623801600000000000,435.36,88.12,17.784167,2247.43,43.0,83.571292,36.037500,32.233333,32.198611,32.381944,394.777778,398.430556,396.736111,73.000000
1623888000000000000,455.25,88.43,17.896250,1416.76,43.0,83.633875,36.036111,32.219444,32.194444,32.362500,392.361111,396.583333,395.194444,73.000000
1623974400000000000,433.74,88.32,17.996143,2166.77,43.0,83.449194,36.037361,32.212500,32.205556,32.355556,391.611111,396.388889,395.000000,73.000000


In [49]:
df = df.dropna(subset=['LIQ_RATE'])
df

,LIQ_RATE,WATER_CUT,PLIN,QGAS,FREQ_HZ,ACTIV_POWER,PINP,I_B,I_A,I_C,U_AB,U_BC,U_CA,COS_PHI
date,,,,,,,,,,,,,,
1616284800000000000,388.84,87.46,17.755278,1853.77,40.0,68.041444,37.168472,29.459722,29.461111,29.552778,395.416667,398.861111,398.958333,71.000000
1616371200000000000,390.57,88.04,17.674028,1809.27,40.0,68.106472,37.155556,29.462500,29.463889,29.563889,393.916667,399.166667,397.638889,71.000000
1616457600000000000,395.10,87.95,17.800694,1822.53,40.0,68.024792,37.147778,29.445833,29.441667,29.537500,395.486111,400.666667,398.083333,71.000000
1616544000000000000,389.34,88.02,17.786528,1855.33,40.0,68.118278,37.144306,29.451389,29.444444,29.534722,395.138889,400.083333,397.805556,71.000000
1616630400000000000,391.28,88.22,17.830278,1827.74,40.0,68.120028,37.135000,29.454167,29.445833,29.550000,396.805556,399.916667,398.111111,71.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1623801600000000000,435.36,88.12,17.784167,2247.43,43.0,83.571292,36.037500,32.233333,32.198611,32.381944,394.777778,398.430556,396.736111,73.000000
1623888000000000000,455.25,88.43,17.896250,1416.76,43.0,83.633875,36.036111,32.219444,32.194444,32.362500,392.361111,396.583333,395.194444,73.000000
1623974400000000000,433.74,88.32,17.996143,2166.77,43.0,83.449194,36.037361,32.212500,32.205556,32.355556,391.611111,396.388889,395.000000,73.000000


In [35]:
interpolator = interp1d(df.index, df['I_A'], kind='linear')

In [36]:
min_time = df.index.min()
max_time = df.index.max()
new_time = pd.date_range(min_time, max_time, freq='1D')


1624147200000000000

In [37]:
interpolatorib =  interp1d(df.index, df['I_B'], kind='linear')

In [57]:
interpolatoric =  interp1d(df.index, df['I_C'], kind='linear')
interpolatoriacvpw =  interp1d(df.index, df['ACTIV_POWER'], kind='linear')
interpolatorpinp =  interp1d(df.index, df['PINP'], kind='linear')
interpolatorcos =  interp1d(df.index, df['COS_PHI'], kind='linear')
new_ia = interpolator(new_time.astype(int))
new_ib = interpolatorib(new_time.astype(int))
new_ic = interpolatoric(new_time.astype(int))
new_pinp = interpolatorpinp(new_time.astype(int))
new_cos = interpolatorcos(new_time.astype(int))
new_acvpw = interpolatoriacvpw(new_time.astype(int))
interpolatorliq =  interp1d(df.index,df['LIQ_RATE'],kind='linear')
new_liq=interpolatorliq(new_time.astype(int))

In [58]:

interpolated_data = pd.DataFrame({'date': new_time,'LIQ_RATE':new_liq,'I_A': new_ia,'I_B':new_ib,'I_C': new_ic,'PINP':new_pinp,'COS_PHI':new_cos,'ACTIV_POWER':new_acvpw})

In [59]:
interpolated_data['date'] = pd.to_datetime(interpolated_data['date'])

In [60]:
interpolated_data

,date,LIQ_RATE,I_A,I_B,I_C,PINP,COS_PHI,ACTIV_POWER
0,2021-03-21,388.84,29.461111,29.459722,29.552778,37.168472,71.000000,68.041444
1,2021-03-22,390.57,29.463889,29.462500,29.563889,37.155556,71.000000,68.106472
2,2021-03-23,395.10,29.441667,29.445833,29.537500,37.147778,71.000000,68.024792
3,2021-03-24,389.34,29.444444,29.451389,29.534722,37.144306,71.000000,68.118278
4,2021-03-25,391.28,29.445833,29.454167,29.550000,37.135000,71.000000,68.120028
...,...,...,...,...,...,...,...,...
87,2021-06-16,435.36,32.198611,32.233333,32.381944,36.037500,73.000000,83.571292
88,2021-06-17,455.25,32.194444,32.219444,32.362500,36.036111,73.000000,83.633875
89,2021-06-18,433.74,32.205556,32.212500,32.355556,36.037361,73.000000,83.449194
90,2021-06-19,433.52,32.208333,32.220833,32.373611,36.035694,73.111111,83.525903


In [61]:
interpolated_data.to_csv('интерполированные_данные.csv', index=False)